# Experiment 1: Augmentation

In [1]:
import os
import sys
import pickle
from glob import glob
from time import time
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score

# data processing
import cv2
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline

# pytorch
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models, utils
torch.manual_seed(0)
print(f'Pytorch version: {torch.__version__}')

# device setting
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'This notebook use {device}')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Pytorch version: 1.6.0
This notebook use cuda:0


In [2]:
# 파일 경로 사용자 정의
class path:
    train = '../input/data/train'
    train_img = f'{train}/images'
    train_df = f'{train}/train.csv'
    test = '../input/data/eval'
    test_img = f'{test}/images'
    test_df = f'{test}/info.csv'

In [3]:
BATCH_SIZE=16
NUM_WORKERS=2
LEARNING_RATE=1e-4
EPOCHS=3

## 1. Dataset Construction

In [4]:
class MaskDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def set_transform(self, transform):
        self.transform = transform
    
    def __getitem__(self, index):
        current_data = self.data.iloc[index]
        target = current_data.target
        image = Image.open(current_data.path)
        
        if self.transform:
            image = self.transform(image)
        
        return image, target
        
    def __len__(self):
        return len(self.data)

## 2. Augmentation Methods

- RandomHorizontalFlip
- CenterCrop
- RandomCrop
- Resize
- Normalize: ToTensor() 이후에 사용
- Color Jitter - Brightness
- Color Jitter - Contrast
- Color Jitter - Saturation
- ToTensor
- GrayScale
- FiveCrop, TenCrop
- GaussianBlur

In [5]:
# transforms의 종류
dir(transforms)[:-13]

['CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'FiveCrop',
 'Grayscale',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandomAffine',
 'RandomApply',
 'RandomChoice',
 'RandomCrop',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomOrder',
 'RandomPerspective',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSizedCrop',
 'RandomVerticalFlip',
 'Resize',
 'Scale',
 'TenCrop',
 'ToPILImage',
 'ToTensor']

### 2.0. No Augmentation

In [17]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [18]:
valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [19]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검증 데이터셋으로 분리
n_train = int(len(dataset)*0.8)
n_valid = len(dataset)-n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms)
valid_dataset.dataset.set_transform(valid_transforms)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, shuffle=False)

### 2.1. Augmentation 1
- CenterCrop (224, 224)
- RandomHorizontalFlip
- RandomRotation
- Color Jitter
- ToTensor
- Normalize

In [6]:
train_transforms1 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation([-5, +5]),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [7]:
valid_transforms1 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [8]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검증 데이터셋으로 분리
n_train = int(len(dataset)*0.8)
n_valid = len(dataset)-n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms1)
valid_dataset.dataset.set_transform(valid_transforms1)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, shuffle=False)

### 2.2. Augmentation 2
- CenterCrop (384, 384)
- Resize (224, 224)
- RandomHorizontalFlip
- RandomRotation
- Color Jitter
- ToTensor
- Normalize

In [39]:
train_transforms2 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation([-5, +5]),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [42]:
valid_transforms2 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [16]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검증 데이터셋으로 분리
n_train = int(len(dataset)*0.8)
n_valid = len(dataset)-n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms2)
valid_dataset.dataset.set_transform(valid_transforms2)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, shuffle=False)

### 2.3. Augmentation 3
- CenterCrop (384, 384)
- Resize (224, 224)
- GrayScale
- RandomHorizontalFlip
- RandomRotation
- Color Jitter
- ToTensor
- Normalize

In [20]:
train_transforms3 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation([-5, +5]),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.24]),
])

In [21]:
valid_transforms3 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.24]),
])

In [22]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검증 데이터셋으로 분리
n_train = int(len(dataset)*0.8)
n_valid = len(dataset)-n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms3)
valid_dataset.dataset.set_transform(valid_transforms3)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, shuffle=False)

### 2.4. Augmentation 4

- CenterCrop (384, 384)
- Resize (224, 224)
- RandomHorizontalFlip
- Color Jitter
- ToTensor
- Normalize

In [40]:
train_transforms4 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [43]:
valid_transforms4 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [44]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검증 데이터셋으로 분리
n_train = int(len(dataset)*0.8)
n_valid = len(dataset)-n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms4)
valid_dataset.dataset.set_transform(valid_transforms4)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, shuffle=False)

### 2.5. Augmentation 5

- RandomResizedCrop(224)
- RandomHorizontalFlip
- Color Jitter
- ToTensor
- Normalize

In [53]:
train_transforms5 = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [54]:
valid_transforms5 = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [55]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검증 데이터셋으로 분리
n_train = int(len(dataset)*0.8)
n_valid = len(dataset)-n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms5)
valid_dataset.dataset.set_transform(valid_transforms5)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, shuffle=False)

### 2.6. Augmentation 6
- CenterCrop (384, 384)
- Resize (224, 224)
- RandomHorizontalFlip
- Color Jitter
- ToTensor
- Normalize
- AddGaussianNoise

In [59]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [60]:
train_transforms6 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
    AddGaussianNoise(0., 1.,)
])

In [61]:
valid_transforms6 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [62]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검증 데이터셋으로 분리
n_train = int(len(dataset)*0.8)
n_valid = len(dataset)-n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms6)
valid_dataset.dataset.set_transform(valid_transforms6)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, shuffle=False)

### 2.6. Augmentation 7
- CenterCrop (384, 384)
- Resize (224, 224)
- Color Jitter
- ToTensor
- Normalize
- AddGaussianNoise

In [38]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [39]:
train_transforms7 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
    AddGaussianNoise(0., 1.,)
])

In [41]:
valid_transforms7 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [42]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검증 데이터셋으로 분리
n_train = int(len(dataset)*0.8)
n_valid = len(dataset)-n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms7)
valid_dataset.dataset.set_transform(valid_transforms7)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, shuffle=False)

## 3. Compare Performance

In [33]:
def test_eval(model, valid_dataset, channel):
    model.eval()
    with torch.no_grad():
        y_true, y_pred = [], []
        for image, label in tqdm(valid_dataset):
            X = image.float().to(device)
            y = label.item()
            _, pred = torch.max(model(X), 1)
            pred = pred.item()
            y_true.append(y)
            y_pred.append(pred)
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')
        accuracy = accuracy_score(y_true, y_pred)
    model.train()
    return f1, accuracy

In [34]:
def train_model(train, test, channel, model, criterion, optimizer, print_every=1):
    print(f"============ Training Starts! ============")
    best_accuracy = 0
    for epoch in range(EPOCHS):
        loss_sum = 0
        for images, label in tqdm(train):
            X = images.float().to(device)
            y = label.to(device)
            
            y_pred = model(X)
            loss = criterion(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
          
            loss_sum += loss
          
        if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
            loss_avg = loss_sum / len(train)
            f1, accuracy = test_eval(model, test, channel)
            print(f">> epoch:[{epoch+1}/{EPOCHS}] cost:{loss_avg:5.3f} test_accuracy:{accuracy:5.3f} test_f1_score:{f1:5.3f}") 

    print(f"============ Training Done! ============")  

### 3.0. No Augmentation

In [35]:
model = models.resnet18(pretrained=False)
n_features = model.fc.in_features
model.fc = nn.Linear(n_features, 18)
model = model.cuda()

In [36]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss().to(device)

In [37]:
train_model(train_loader, valid_loader, 3, model, criterion, optimizer)

============ Training Starts! ============



>> epoch:[1/3] cost:1.306 test_accuracy:0.636 test_f1_score:0.373



>> epoch:[2/3] cost:0.625 test_accuracy:0.752 test_f1_score:0.540



>> epoch:[3/3] cost:0.452 test_accuracy:0.821 test_f1_score:0.726
============ Training Done! ============


### 3.1. Augmentation 1

In [11]:
model1 = models.resnet18(pretrained=False)
n_features = model1.fc.in_features
model1.fc = nn.Linear(n_features, 18)
model1 = model1.cuda()

In [12]:
optimizer1 = optim.Adam(model1.parameters(), lr=LEARNING_RATE)
criterion1 = nn.CrossEntropyLoss().to(device)

In [13]:
train_model(train_loader, valid_loader, 3, model1, criterion1, optimizer1)

============ Training Starts! ============



>> epoch:[1/3] cost:1.096 test_accuracy:0.709 test_f1_score:0.511



>> epoch:[2/3] cost:0.589 test_accuracy:0.821 test_f1_score:0.714



>> epoch:[3/3] cost:0.441 test_accuracy:0.837 test_f1_score:0.657
============ Training Done! ============


### 3.2. Augmentation 2

In [17]:
model2 = models.resnet18(pretrained=False)
n_features = model2.fc.in_features
model2.fc = nn.Linear(n_features, 18)
model2 = model2.cuda()

In [18]:
optimizer2 = optim.Adam(model2.parameters(), lr=LEARNING_RATE)
criterion2 = nn.CrossEntropyLoss().to(device)

In [19]:
train_model(train_loader, valid_loader, 3, model2, criterion2, optimizer2)

============ Training Starts! ============



>> epoch:[1/3] cost:1.083 test_accuracy:0.699 test_f1_score:0.529



>> epoch:[2/3] cost:0.522 test_accuracy:0.838 test_f1_score:0.686



>> epoch:[3/3] cost:0.353 test_accuracy:0.843 test_f1_score:0.711
============ Training Done! ============


### 3.3. Augmentation 3

In [34]:
model3 = models.resnet18(pretrained=False)
n_features = model3.fc.in_features
model3.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model3.fc = nn.Linear(n_features, 18)
model3 = model3.cuda()

In [35]:
optimizer3 = optim.Adam(model3.parameters(), lr=LEARNING_RATE)
criterion3 = nn.CrossEntropyLoss().to(device)

In [36]:
train_model(train_loader, valid_loader, 1, model3, criterion3, optimizer3)

============ Training Starts! ============



>> epoch:[1/3] cost:1.187 test_accuracy:0.685 test_f1_score:0.498



>> epoch:[2/3] cost:0.533 test_accuracy:0.843 test_f1_score:0.725



>> epoch:[3/3] cost:0.346 test_accuracy:0.707 test_f1_score:0.661
============ Training Done! ============


### 3.4. Augmentation 4

In [45]:
model4 = models.resnet18(pretrained=False)
n_features = model4.fc.in_features
model4.fc = nn.Linear(n_features, 18)
model4 = model4.cuda()

In [46]:
optimizer4 = optim.Adam(model4.parameters(), lr=LEARNING_RATE)
criterion4 = nn.CrossEntropyLoss().to(device)

In [47]:
train_model(train_loader, valid_loader, 3, model4, criterion4, optimizer4)

============ Training Starts! ============



>> epoch:[1/3] cost:1.089 test_accuracy:0.770 test_f1_score:0.559



>> epoch:[2/3] cost:0.540 test_accuracy:0.837 test_f1_score:0.696



>> epoch:[3/3] cost:0.386 test_accuracy:0.868 test_f1_score:0.729
============ Training Done! ============


### 3.5. Augmentation 5

In [56]:
model5 = models.resnet18(pretrained=False)
n_features = model5.fc.in_features
model5.fc = nn.Linear(n_features, 18)
model5 = model5.cuda()

In [57]:
optimizer5 = optim.Adam(model5.parameters(), lr=LEARNING_RATE)
criterion5 = nn.CrossEntropyLoss().to(device)

In [58]:
train_model(train_loader, valid_loader, 3, model5, criterion5, optimizer5)

============ Training Starts! ============



>> epoch:[1/3] cost:1.119 test_accuracy:0.748 test_f1_score:0.556



>> epoch:[2/3] cost:0.557 test_accuracy:0.675 test_f1_score:0.570



>> epoch:[3/3] cost:0.381 test_accuracy:0.766 test_f1_score:0.653
============ Training Done! ============


### 3.6. Augmentation 6

In [64]:
model6 = models.resnet18(pretrained=False)
n_features = model6.fc.in_features
model6.fc = nn.Linear(n_features, 18)
model6 = model6.cuda()

In [65]:
optimizer6 = optim.Adam(model6.parameters(), lr=LEARNING_RATE)
criterion6 = nn.CrossEntropyLoss().to(device)

In [66]:
train_model(train_loader, valid_loader, 3, model6, criterion6, optimizer6)

============ Training Starts! ============



>> epoch:[1/3] cost:1.077 test_accuracy:0.761 test_f1_score:0.562



>> epoch:[2/3] cost:0.534 test_accuracy:0.847 test_f1_score:0.732



>> epoch:[3/3] cost:0.370 test_accuracy:0.856 test_f1_score:0.772
============ Training Done! ============


### 3.7. Augmentation 7

In [43]:
model7 = models.resnet18(pretrained=False)
n_features = model7.fc.in_features
model7.fc = nn.Linear(n_features, 18)
model7 = model7.cuda()

In [44]:
optimizer7 = optim.Adam(model7.parameters(), lr=LEARNING_RATE)
criterion7 = nn.CrossEntropyLoss().to(device)

In [45]:
train_model(train_loader, valid_loader, 3, model7, criterion7, optimizer7)

============ Training Starts! ============



>> epoch:[1/3] cost:1.078 test_accuracy:0.721 test_f1_score:0.477



>> epoch:[2/3] cost:0.536 test_accuracy:0.819 test_f1_score:0.632



>> epoch:[3/3] cost:0.375 test_accuracy:0.878 test_f1_score:0.745
============ Training Done! ============
